### Установка библиотек, импорт файла с данными


In [ ]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00


In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval
import re
import string

In [ ]:
df = pd.read_excel('twitter_dialogs_labeled_data.xlsx')

###Выбор метки:

In [ ]:
# label_name = 'is_negative'
# label_name = 'has_gratitude'
label_name = 'dialog_type'

###Проверка меток и преобразование их для случая жалоб/вопросов:

In [ ]:
if label_name=='dialog_type':
  df[label_name].replace({'Жалоба': 0, 'Вопрос': 1}, inplace=True)
df = df[df[label_name].isin([0, 1])].reset_index(drop=True)
df[label_name] = df[label_name].astype(int)
df[label_name].value_counts()

0    866
1    228
Name: dialog_type, dtype: int64

###Предобработка

In [ ]:
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def remove_nicknames(text):
    clean = re.compile('@\w+')
    return re.sub(clean, '', text)

def remove_urls(text):
    clean = re.compile(r'http\S+|www\S+')
    return re.sub(clean, '', text)

In [ ]:
df['text'] = df['text'].apply(remove_html_tags)
df['text'] = df['text'].apply(remove_nicknames)
df['text'] = df['text'].apply(remove_urls)

###Объединение отдельно реплик клиента в одну строку:

In [ ]:
df['text'] = df['text'].apply(lambda x: ' '.join([line for line in x.split('\n') if 'Agent' not in line]))
df['text'] = df['text'].str.replace('\tCustomer\t', '')
df['text']

0       \tCustomer:\tSo neither my iPhone nor my Apple...
1       \tCustomer:\t hi team! i m planning to get App...
2       \tCustomer:\t Where do I write to address a cu...
3       \tCustomer:\t  Wow, expected 4 packages yester...
4       \tCustomer:\t I'd rather you spent some time m...
                              ...                        
1089    \tCustomer:\t flight 112 from ORD-BOS was dela...
1090    \tCustomer:\t Hi. Pe-ordered #StarWarsBattlefr...
1091    \tCustomer:\tHolding for over an hour, .\n\tAg...
1092    \tCustomer:\t saludos,es que tengo un problema...
1093    \tCustomer:\t does Mpesa have any delays? I've...
Name: text, Length: 1094, dtype: object

###Выбор модели и токенизатора

In [ ]:
from torch.utils.data import Dataset
from transformers import AutoTokenizer

tokenizer_name = 'bert-base-uncased'
# model_name = "SoooSlooow/tweet-bert-uncased"
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
torch.manual_seed(42)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

###Обучение с помощью кросс-валидации:

In [ ]:
class ClassificationTorchDataset(Dataset):

    def __init__(self, df):
        self.df = df

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        model_inputs = tokenizer(
            self.df.iloc[idx, :]['text'], max_length=512,
            truncation=True
        )
        model_inputs['label'] = self.df.iloc[idx, :][label_name]
        return model_inputs

In [ ]:
import evaluate

f1 = evaluate.load("f1")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(
    output_dir="model_dir",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-6,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    num_train_epochs=15,
    metric_for_best_model='f1',
    greater_is_better=True,
    push_to_hub=False,
    load_best_model_at_end=True
)

In [ ]:
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        logits = model(**inputs).logits
    return np.array(logits.argmax(axis=1).cpu())

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
import evaluate

n = 5
skf = StratifiedKFold(n_splits=n, random_state=42, shuffle=True)
f1_score_results_0 = []
f1_score_results_1 = []

for train_index, test_index in skf.split(df, df[label_name]):

      model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
      model.to(device)

      f1 = evaluate.load("f1")

      test_df = df.iloc[test_index]
      train_df, eval_df = train_test_split(df.iloc[train_index], test_size=0.25, random_state=42)

      train_dataset = ClassificationTorchDataset(train_df)
      eval_dataset = ClassificationTorchDataset(eval_df)

      trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
        )

      trainer.train()

      labels = test_df[label_name].values.astype('int')
      predictions = predict(test_df['text'].values.tolist())

      precision_only, recall_only, f1_only, support_only = precision_recall_fscore_support(labels, predictions)

      f1_score_results_0.append(f1_only[0])
      f1_score_results_1.append(f1_only[1])

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,No log,0.516477,0.000000
2,No log,0.492063,0.000000
3,No log,0.474970,0.000000
4,No log,0.455955,0.210526
5,No log,0.440709,0.338462
6,No log,0.444448,0.555556
7,No log,0.419852,0.400000
8,No log,0.421839,0.577778
9,No log,0.413590,0.577778
10,No log,0.429715,0.554455


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1
1,No log,0.512697,0.000000
2,No log,0.494115,0.000000
3,No log,0.488300,0.039216
4,No log,0.484717,0.203390
5,No log,0.483421,0.366197
6,No log,0.486159,0.352941
7,No log,0.493492,0.405063
8,No log,0.503323,0.458333
9,No log,0.494962,0.433735
10,No log,0.502280,0.466667


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1
1,No log,0.546895,0.000000
2,No log,0.521448,0.000000
3,No log,0.510949,0.000000
4,No log,0.469602,0.169492
5,No log,0.451666,0.382353
6,No log,0.447479,0.352941
7,No log,0.428044,0.528736
8,No log,0.422568,0.545455
9,No log,0.426397,0.565217
10,No log,0.425163,0.565217


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1
1,No log,0.571254,0.000000
2,No log,0.548913,0.000000
3,No log,0.524733,0.000000
4,No log,0.521943,0.000000
5,No log,0.457040,0.356164
6,No log,0.442438,0.476190
7,No log,0.435668,0.591837
8,No log,0.424546,0.511628
9,No log,0.433061,0.517647
10,No log,0.412163,0.586957


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1
1,No log,0.526466,0.000000
2,No log,0.491055,0.000000
3,No log,0.467888,0.039216
4,No log,0.448313,0.245614
5,No log,0.463985,0.275862
6,No log,0.458630,0.562500
7,No log,0.410400,0.500000
8,No log,0.404392,0.561798
9,No log,0.402302,0.549451
10,No log,0.406749,0.561798


###Результаты:

In [ ]:
if label_name = 'dialog_type':
  mean0 = sum(f1_score_results_0) / len(f1_score_results_0)
  mean1 = sum(f1_score_results_1) / len(f1_score_results_1)
  print(f'f1 score: {(mean0 + mean1)/2}')
else:
  print(f'f1 score: {f1_score_results_1}')

0.7056351677486454